# make a very simple LSTM model with keras

see how long this will take to run for 7t data with a not too complicated model and what kind of resources we are talking about - single run single subject..

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import StandardScaler
import pandas as pd
import nibabel as nb
import hrf_tools
#import hcp_utils as hcp
#from analysis import plot_59k_results

#import matplotlib.pyplot as plt
from analysis import load_data

/opt/miniconda-latest/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [2]:
print(np.__version__)

1.19.5


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13014453756933303561
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7611585664
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9254048372183658020
physical_device_desc: "device: 0, name: GeForce RTX 2080, pci bus id: 0000:02:00.0, compute capability: 7.5"
]


In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
subject=100610
feature='as_scores'
n_movies=3
load_data(subject,feature,n_movies)

### load data

In [6]:
subject=100610
feature='rms'
n_movies=3
scaler = StandardScaler()

stim = ['tfMRI_MOVIE1_7T_AP','tfMRI_MOVIE2_7T_PA','tfMRI_MOVIE3_7T_PA','tfMRI_MOVIE4_7T_AP']
stim_feat = ['7T_MOVIE1_CC1_v2', '7T_MOVIE2_HO1_v2', '7T_MOVIE3_CC2_v2', '7T_MOVIE4_HO2_v2']
    
i=0
X = np.load(f'../sourcedata/data/HCP_7T_movie_FIX/features/{stim_feat[i]}_rms.npy')
X.shape#plt.plot(X[0:1000])

(9211, 1)

In [7]:
X  = np.load(f'../sourcedata/data/HCP_7T_movie_FIX/features/{stim_feat[i]}_as_embed.npy')

In [8]:
X.shape

(1918, 1024)

In [9]:
from scipy.signal import resample

In [10]:
X = resample(X, 921, axis=0) #resample to 1hz for now 

In [11]:
X_hrf = hrf_tools.apply_optimal_hrf_10hz(X,1)

In [12]:
#plt.plot(X[:,1])

In [13]:
img = nb.load('../sourcedata/data/parcellations/parcellated/sub100610_tfMRI_MOVIE1_7T_AP.ptseries.nii')
Y = img.get_fdata()
Y.shape

(921, 360)

In [14]:
img = nb.load('../sourcedata/data/parcellations/parcellated/sub100610_tfMRI_MOVIE2_7T_PA.ptseries.nii')
Y_test = img.get_fdata()
Y_test.shape

(918, 360)

In [15]:
i=1
X_test = np.load(f'../sourcedata/data/HCP_7T_movie_FIX/features/{stim_feat[i]}_rms.npy')
X_test = resample(X, 918, axis=0) #resample to 1hz for now 
X_test_hrf = hrf_tools.apply_optimal_hrf_10hz(X_test,1)
X_test.shape#plt.plot(X[0:1000])

(918, 1024)

In [16]:
# im_file = f'../sourcedata/data/HCP_7T_movie_FIX/brain/HCP_7T_movie_FIX/100610/MNINonLinear/Results/{stim[i]}/{stim[i]}_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
# img = nb.load(im_file)
# img_y = img.get_fdata()
# Y = img_y
#Y is time x brain
# Y.shape

In [17]:
#X is time x embed
X_hrf.shape

(921, 1024)

### design a simple(?) lstm model

In [18]:
# model = keras.Sequential(
#     [
#         layers.Dense(2, activation="relu", name="layer1"),
#         layers.Dense(3, activation="relu", name="layer2"),
#         layers.Dense(4, name="layer3"),
#     ]
# )
# # Call model on a test input
# x = tf.ones((3, 3))
# y = model(x)

In [19]:
# https://github.com/tensorflow/tensorflow/issues/37942
# Hi,

# so the following workaround has been found thanks to @DietmarKracht. Now I am able to train LSTMs on my GPU without the error from the first post in this issue. 🎉 🎉 🎉

# # To train the LSTM-model on a GPU on my plattform (Windows 10, tf 2.1.0), the parameter batch_input_shape must be specified during the model creation in the very first layer and the parameter input_shape must be omitted. The first layer of the LSTM-model should looks this:
    
# model = Sequential()
# model.add(LSTM(100, 
#     batch_input_shape=(batch_size, n_timesteps, n_features), 
#     return_sequences=True))  # omit return_sequences if no other LSTM-layer follows
# [...]

In [20]:
# Batch size
batch_size = 64
time_steps = 10
input_units = 1024
input_shape = (time_steps,input_units)

model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
#model.add(layers.Embedding(input_dim=1024, output_dim=1024))

#model.add(keras.Input(shape=input_shape)) #omit beacuse of bug
# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(1024,batch_input_shape=(batch_size, time_steps, input_units)))
model.add(layers.Dense(1024))
model.add(layers.Dense(512))
model.add(layers.Dense(360))

#model.add(layers.UpSampling1D(size=10))
#model.add(layers.Dense(17049))
#model.add(layers.Dense(128))
model.add(layers.Dense(360))
#model.add(layers.Dense(360))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (64, 1024)                8392704   
_________________________________________________________________
dense (Dense)                (64, 1024)                1049600   
_________________________________________________________________
dense_1 (Dense)              (64, 512)                 524800    
_________________________________________________________________
dense_2 (Dense)              (64, 360)                 184680    
_________________________________________________________________
dense_3 (Dense)              (64, 360)                 129960    
Total params: 10,281,744
Trainable params: 10,281,744
Non-trainable params: 0
_________________________________________________________________


In [21]:
# #dataset = tf.data.Dataset.from_tensor_slices((np.expand_dims(x_train,0), np.expand_dims(y_train,0)))
# dataset = tf.data.Dataset.from_tensor_slices((X_hrf, Y))

# dataset.element_spec

In [22]:
#https://keras.io/api/preprocessing/timeseries/
dataset = tf.keras.preprocessing.timeseries_dataset_from_array(data = X_hrf, 
                                                     targets = Y, 
                                                    sequence_length=time_steps,
                                                     batch_size = batch_size)

In [23]:
dataset

<BatchDataset shapes: ((None, None, 1024), (None, 360)), types: (tf.float32, tf.float64)>

In [24]:


# # Buffer size to shuffle the dataset
# # (TF data is designed to work with possibly infinite sequences,
# # so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# # it maintains a buffer in which it shuffles elements).
# BUFFER_SIZE = 10000

# dataset = (
#     dataset
#     .shuffle(BUFFER_SIZE)
#     .batch(batch_size, drop_remainder=True)
#     .prefetch(tf.data.experimental.AUTOTUNE))

# dataset

In [25]:
#simple_rnn = layers.SimpleRNN(18, return_sequences=True, return_state=True)

In [26]:
#whole_sequence_output, final_state = simple_rnn(np.expand_dims(x_train,0))

In [29]:
model.compile(
    loss=keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"),
    optimizer="adam",
    metrics=["accuracy", 'mean_squared_error'],
)

In [31]:
dataset

<BatchDataset shapes: ((None, None, 1024), (None, 360)), types: (tf.float32, tf.float64)>

In [28]:
history = model.fit(dataset,epochs = 20)

Epoch 1/20


InternalError:    Failed to call ThenRnnForward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 1024, 1024, 1, 10, 64, 1024] 
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/PartitionedCall]] [Op:__inference_train_function_2814]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
model.save('../outputs/model.pb')

In [ ]:
model = keras.models.load_model('../outputs/model.pb')

In [72]:
dataset_test = tf.keras.preprocessing.timeseries_dataset_from_array(data = X_test_hrf, 
                                                     targets = Y_test, 
                                                    sequence_length=time_steps,
                                                     batch_size = batch_size)

In [74]:
model.evaluate(dataset_test)

15/15 [==============================] - 0s 14ms/step - loss: 2109020.0000 - accuracy: 0.0000e+00 - mean_squared_error: 2109020.0000


[2109020.0, 0.0, 2109020.0]